In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob

In [4]:
data = []
list = [pd.read_json(filename, lines=True) for filename in glob.glob("../week4/data/return-data/*.txt")]
df = pd.concat(list, axis=0)
df.head()

,basket,returnLabel,totalAmount,transactionId,zipCode
0,"[3, 1]",0,106,3520406327,7667
1,[1],0,50,6030070379,7647
2,"[1, 1, 5, 4, 1]",0,175,6579182382,2633
3,"[1, 2, 0, 1]",0,272,2779338441,4309
4,[2],0,14,3198368336,4659


In [5]:
df.drop(['transactionId','zipCode'],axis=1,inplace=True)
df.head()

,basket,returnLabel,totalAmount
0,"[3, 1]",0,106
1,[1],0,50
2,"[1, 1, 5, 4, 1]",0,175
3,"[1, 2, 0, 1]",0,272
4,[2],0,14


In [6]:
def categories(c, value):
    return c.count(value)

df['c_0'] = df['basket'].apply(lambda x: categories(x, 0))
df['c_1'] = df['basket'].apply(lambda x: categories(x, 1))
df['c_2'] = df['basket'].apply(lambda x: categories(x, 2))
df['c_3'] = df['basket'].apply(lambda x: categories(x, 3))
df['c_4'] = df['basket'].apply(lambda x: categories(x, 4))
df['c_5'] = df['basket'].apply(lambda x: categories(x, 5))

df.drop(['basket'],axis=1,inplace=True)

df.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,106,0,1,0,1,0,0
1,0,50,0,1,0,0,0,0
2,0,175,0,3,0,0,1,1
3,0,272,1,2,1,0,0,0
4,0,14,0,0,1,0,0,0


In [7]:
from sklearn import preprocessing

def normalize(data, target):
    norm = preprocessing.normalize(data, axis=0)
    df_concat = np.concatenate((target, norm), axis=1)

    return pd.DataFrame(df_concat)

In [8]:
df_norm = normalize(df[['totalAmount', 'c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5']], df[['returnLabel']])
df_norm.columns = ['returnLabel', 'totalAmount', 'c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5']
df_norm.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0.0,0.001905,0.000000,0.004752,0.00000,0.004611,0.000000,0.00000
1,0.0,0.000899,0.000000,0.004752,0.00000,0.000000,0.000000,0.00000
2,0.0,0.003145,0.000000,0.014257,0.00000,0.000000,0.004447,0.00481
3,0.0,0.004889,0.004764,0.009505,0.00473,0.000000,0.000000,0.00000
4,0.0,0.000252,0.000000,0.000000,0.00473,0.000000,0.000000,0.00000


In [9]:
from sklearn.model_selection import train_test_split

X = df_norm.drop(['returnLabel'],axis=1)
y = df_norm['returnLabel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=101)

In [10]:
X_test_norm = preprocessing.normalize(X_test)
X_test_norm

array([[0.39711915, 0.        , 0.39625323, ..., 0.19223008, 0.37083059,
        0.40108332],
       [0.27895394, 0.21496773, 0.        , ..., 0.20804172, 0.80266559,
        0.43407392],
       [0.06542382, 0.86717174, 0.28832514, ..., 0.27974416, 0.        ,
        0.        ],
       ...,
       [0.36639396, 0.29432935, 0.58716778, ..., 0.28484641, 0.        ,
        0.59432501],
       [0.55776807, 0.55796503, 0.55655185, ..., 0.        , 0.26042242,
        0.        ],
       [0.38919369, 0.3738143 , 0.37286752, ..., 0.        , 0.        ,
        0.75482511]])

In [11]:
X_train_norm = preprocessing.normalize(X_train)
X_train_norm

array([[0.16051026, 0.        , 0.98703417, ..., 0.        , 0.        ,
        0.        ],
       [0.25620601, 0.        , 0.        , ..., 0.9666222 , 0.        ,
        0.        ],
       [0.03797113, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.37122128, 0.4731172 , 0.47191892, ..., 0.        , 0.44164176,
        0.        ],
       [0.31546898, 0.43556666, 0.        , ..., 0.84306643, 0.        ,
        0.        ],
       [0.41162489, 0.        , 0.39579179, ..., 0.38401245, 0.37039876,
        0.20030813]])

In [12]:
import math

def euclideanDistance(feature1, feature2, k):
    distance = 0
    for i in range(k):
        distance += pow((feature1[i] - feature2[i]), 2)
    return math.sqrt(distance)

In [13]:
# Teste die Methode euclideanDistance
data1 = [2, 2, 2]
data2 = [4, 4, 4]
distance = euclideanDistance(data1, data2, 3)
distance

3.4641016151377544

In [14]:
import operator

def getNeighbors(trainSet, testFeature, k):
    distances = []
    length = len(testFeature) - 1
    for i in range(len(trainSet)):
        distance = euclideanDistance(testFeature, trainSet[i], length)
        distances.append((trainSet[i], distance))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

In [15]:
# Teste die Methode getNeighbors
trainSet = [[2, 2, 2], [4, 4, 4], [6, 6, 6]]
testFeature = [5, 5, 5]
k = 2
neighbors = getNeighbors(trainSet, testFeature, k)
neighbors

[[4, 4, 4], [6, 6, 6]]

In [16]:
def getResponse(neighbors):
    classVotes = {}
    for i in range(len(neighbors)):
        response = neighbors[i][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse = True)
    return sortedVotes[0][0]

In [17]:
# Teste die Methode getResponse
neighbors = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
response = getResponse(neighbors)
response

3

In [18]:
def customNeighborClassifier(trainSet, testSet, k):
    predictions = []
    for i in range(len(testSet)):
        neighbors = getNeighbors(trainSet, testSet[i], k)
        result = getResponse(neighbors)
        predictions.append(result)
    return predictions

In [19]:
# Teste die Methode customNeighborClassifier
trainSet = [[2, 2, 2], [4, 4, 4], [6, 6, 6]]
testSet = [[5, 5, 5], [8, 8, 8], [7, 7, 7]]
predictions = customNeighborClassifier(trainSet, testSet, 2)
predictions

[4, 6, 6]

In [20]:
neighborPredictions = customNeighborClassifier(X_train_norm, X_test_norm, 5)
neighborPredictions

[0.0,
 0.5273375071242068,
 0.0,
 0.0,
 0.0,
 0.6716184351095938,
 0.0,
 0.5799259929225405,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5028281561305366,
 0.0,
 0.0,
 0.4111874832505943,
 0.8641815309542594,
 0.0,
 0.3623036330050757,
 0.0,
 0.0,
 0.4428248560398195,
 0.0,
 0.9795317523134067,
 0.29125058573203366,
 0.6048643383168812,
 0.41272126237239154,
 0.7288504248637465,
 0.0,
 0.0,
 0.7207340667911204,
 0.6396316085887173,
 0.0,
 0.0,
 0.549712378377923,
 0.36260420366417756,
 0.720860383443129,
 0.6324246917082427,
 0.0,
 0.653401176638374,
 0.0,
 0.665980926337075,
 0.9982179144188034,
 0.0,
 0.0,
 0.0,
 0.21047625951108972,
 0.0,
 0.9742028850135986,
 0.0,
 0.9920275703880688,
 0.9795317523134067,
 0.3991828201844832,
 0.9575614094371419,
 0.41141802414685974,
 0.0,
 0.0,
 0.4817764333262797,
 0.6703978121858457,
 0.0,
 0.627104008072555,
 0.0,
 0.7991782792287383,
 0.0,
 0.0,
 0.5816857396428531,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5753013173827428,
 0.0,
 0.27412526505762674,
 0.734306

In [21]:
def getAccuracy(testSet, neighborPredictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == neighborPredictions[i]:
            correct += 1

    accuracy = float(correct)/float(len(testSet)) * 100.00
    return accuracy

In [22]:
# Teste die Methode getAccuracy
testSet = [[1, 1, 1], [2, 2, 2], [3, 3, 3]]
predictions = [1, 1, 1]
accuracy = getAccuracy(testSet, predictions)
accuracy

33.33333333333333

In [23]:
accuracy = getAccuracy(X_test_norm, neighborPredictions)
accuracy

56.892473118279575